In [1]:
import os
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')

import numpy as np
import random
import torch
# %env CUDA_VISIBLE_DEVICES=1

from lib import data_processors, utils

In [2]:
params = {
    'data_dir': '../../data/SST-2',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'sst2',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 5,
    'seed': 1331
}

processors = {
    'cola': data_processors.ColaProcessor,
    'mnli': data_processors.MnliProcessor,
    'mrpc': data_processors.MrpcProcessor,
    'sst2': data_processors.SST2Processor
}

num_labels_task = {
    'cola': 2,
    'mnli': 3,
    'mrpc': 2,
    'sst2': 2
}

random.seed(params['seed'])
np.random.seed(params['seed'])
torch.manual_seed(params['seed'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
from lib.bert import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer

from lib.train_eval import train, evaluate
from lib.train_student import train_student 
from lib.blend_cnn import BlendCNN
from pytorch_pretrained_bert.modeling import (BertConfig, WEIGHTS_NAME, CONFIG_NAME)
from lib.bert import BertForSequenceClassification

Скачиваем предобученный на SST-2 Берт.

In [7]:
processor = processors[params['task_name']]()
num_labels = num_labels_task[params['task_name']]
label_list = processor.get_labels()

tokenizer = BertTokenizer.from_pretrained(
    params['bert_model'], do_lower_case=True)

train_examples = processor.get_train_examples(params['data_dir'])
eval_examples = processor.get_dev_examples(params['data_dir'])

output_model_file = os.path.join(params['output_dir'], WEIGHTS_NAME)
output_config_file = os.path.join(params['output_dir'], CONFIG_NAME)

config = BertConfig(output_config_file)
teacher_model = BertForSequenceClassification(config, num_labels=num_labels)
teacher_model.load_state_dict(torch.load(output_model_file))
teacher_model = teacher_model.to(device)


03/21/2019 16:41:40 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/shakhrayv/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [8]:
result = evaluate(teacher_model, eval_examples, label_list, params, tokenizer)
result

03/21/2019 16:41:46 - INFO - lib.train_eval -   ***** Running evaluation *****
03/21/2019 16:41:46 - INFO - lib.train_eval -     Num examples = 872
03/21/2019 16:41:46 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:06<00:00, 17.64it/s]
03/21/2019 16:41:52 - INFO - lib.train_eval -   ***** Eval results *****
03/21/2019 16:41:52 - INFO - lib.train_eval -     eval_accuracy = 0.9277522935779816
03/21/2019 16:41:52 - INFO - lib.train_eval -     eval_loss = 0.24567339350597575


{'eval_loss': 0.24567339350597575, 'eval_accuracy': 0.9277522935779816}

В качестве студента возьмем модель BlendCNN (код находится в паке lib)

In [9]:
model = BlendCNN(params['max_seq_length'], len(tokenizer.vocab), len(label_list), n_layers=6).to(device)

Общее число параметров в ученике

In [35]:
all_count = 0
for p in model.named_parameters():
    cur_count = 1
    for sh in p[1].size():
        cur_count *= sh
        
    all_count += cur_count
    
print(all_count)

4456598


Общее число параметров в учителе

In [42]:
all_count = 0
for p in teacher_model.named_parameters():
    cur_count = 1
    for sh in p[1].size():
        cur_count *= sh
        
    all_count += cur_count
    
print(all_count)

109483778


In [43]:
print('compresing in {} times'.format(109483778 / 4456598))

compresing in 24.566671259108404 times


Обучаем студента при помощи дистилляции. Далее какое-то время идет обучение, но это не конец.

In [10]:
model, result = train_student(model, teacher_model, train_examples, label_list, params, tokenizer)

Evaluating:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    10520


Iteration:   0%|          | 0/2105 [00:00<?, ?it/s]../lib/train_student.py:111: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_first = KLDivLoss()(F.log_softmax(logits_model / temperature), F.softmax(teacher_logits / temperature))
../lib/train_student.py:111: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_first = KLDivLoss()(F.log_softmax(logits_model / temperature), F.softmax(teacher_logits / temperature))
/home/shakhrayv/.virtualenvs/lenas_py3.7/lib/python3.7/site-packages/torch/nn/functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total los

Epoch: 1


Iteration:   1%|          | 12/2105 [00:00<00:17, 117.38it/s]

...

Iteration:   1%|          | 12/2105 [00:00<00:18, 114.03it/s]

Epoch: 4


Iteration:   1%|          | 12/2105 [00:00<00:18, 116.23it/s]

Epoch: 5


Iteration: 100%|██████████| 2105/2105 [00:18<00:00, 116.37it/s]


In [11]:
evaluate(model, eval_examples, label_list, params, tokenizer)

03/21/2019 16:47:57 - INFO - lib.train_eval -   ***** Running evaluation *****
03/21/2019 16:47:57 - INFO - lib.train_eval -     Num examples = 872
03/21/2019 16:47:57 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 433.20it/s]
03/21/2019 16:47:57 - INFO - lib.train_eval -   ***** Eval results *****
03/21/2019 16:47:57 - INFO - lib.train_eval -     eval_accuracy = 0.5573394495412844
03/21/2019 16:47:57 - INFO - lib.train_eval -     eval_loss = 0.11215483937241616


{'eval_loss': 0.11215483937241616, 'eval_accuracy': 0.5573394495412844}

In [12]:
model, result = train_student(model, teacher_model, train_examples, label_list, params, tokenizer)

Evaluating:   0%|          | 0/2105 [00:00<?, ?it/s]

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    10520


Iteration:   0%|          | 6/2105 [00:00<00:37, 55.66it/s]

Epoch: 1


Iteration:   1%|          | 11/2105 [00:00<00:19, 106.38it/s]

...

Iteration:   0%|          | 9/2105 [00:00<00:24, 86.32it/s]

Epoch: 4


Iteration:   1%|          | 12/2105 [00:00<00:18, 112.70it/s]

Epoch: 5


Iteration: 100%|██████████| 2105/2105 [00:18<00:00, 111.37it/s]


In [20]:
evaluate(model, eval_examples, label_list, params, tokenizer)

03/21/2019 16:56:29 - INFO - lib.train_eval -   ***** Running evaluation *****
03/21/2019 16:56:29 - INFO - lib.train_eval -     Num examples = 872
03/21/2019 16:56:29 - INFO - lib.train_eval -     Batch size = 8
Evaluating: 100%|██████████| 109/109 [00:00<00:00, 384.49it/s]
03/21/2019 16:56:29 - INFO - lib.train_eval -   ***** Eval results *****
03/21/2019 16:56:29 - INFO - lib.train_eval -     eval_accuracy = 0.6009174311926605
03/21/2019 16:56:29 - INFO - lib.train_eval -     eval_loss = 0.10501195025553398


{'eval_loss': 0.10501195025553398, 'eval_accuracy': 0.6009174311926605}

In [38]:
params['num_train_epochs'] = 5
params['learning_rate'] = 2e-3

model, result = train_student(model, teacher_model, train_examples, label_list, params, tokenizer)

***** Running training *****
Num examples: 67349
Batch size:   128
Num steps:    2630


Evaluating:  48%|████▊     | 253/527 [01:31<01:40,  2.72it/s]


Evaluating:  96%|█████████▌| 505/527 [03:04<00:08,  2.70it/s]


Iteration:   0%|          | 0/527 [00:00<?, ?it/s]../lib/train_student.py:111: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_first = KLDivLoss()(F.log_softmax(logits_model / temperature), F.softmax(teacher_logits / temperature))
../lib/train_student.py:111: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  loss_first = KLDivLoss()(F.log_softmax(logits_model / temperature), F.softmax(teacher_logits / temperature))
/home/shakhrayv/.virtualenvs/lenas_py3.7/lib/python3.7/site-packages/torch/nn/functional.py:1906: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss

Epoch: 1


...


Iteration:   1%|          | 6/527 [00:00<00:09, 53.02it/s]

Epoch: 4



Iteration:   1%|          | 6/527 [00:00<00:09, 55.67it/s]

Epoch: 5



Iteration: 100%|██████████| 527/527 [00:09<00:00, 54.05it/s]

In [39]:
evaluate(model, eval_examples, label_list, params, tokenizer)

03/21/2019 17:15:55 - INFO - lib.train_eval -   ***** Running evaluation *****
03/21/2019 17:15:55 - INFO - lib.train_eval -     Num examples = 872
03/21/2019 17:15:55 - INFO - lib.train_eval -     Batch size = 8

Evaluating: 100%|██████████| 109/109 [00:00<00:00, 412.48it/s]03/21/2019 17:15:55 - INFO - lib.train_eval -   ***** Eval results *****
03/21/2019 17:15:55 - INFO - lib.train_eval -     eval_accuracy = 0.7763761467889908
03/21/2019 17:15:55 - INFO - lib.train_eval -     eval_loss = 0.002517868346030559


{'eval_loss': 0.002517868346030559, 'eval_accuracy': 0.7763761467889908}

In [40]:
model, result = train_student(model, teacher_model, train_examples, label_list, params, tokenizer)

***** Running training *****
Num examples: 67349
Batch size:   128
Num steps:    2630



Evaluating:  48%|████▊     | 253/527 [01:32<01:40,  2.71it/s]


Evaluating:  96%|█████████▌| 505/527 [03:05<00:08,  2.70it/s]


Iteration:   1%|          | 3/527 [00:00<00:18, 27.91it/s]

Epoch: 1


...


Iteration:   1%|          | 6/527 [00:00<00:10, 52.00it/s]

Epoch: 4



Iteration:   1%|          | 6/527 [00:00<00:09, 55.01it/s]

Epoch: 5



Iteration: 100%|██████████| 527/527 [00:09<00:00, 54.17it/s]

In [41]:
evaluate(model, eval_examples, label_list, params, tokenizer)

03/21/2019 17:26:22 - INFO - lib.train_eval -   ***** Running evaluation *****
03/21/2019 17:26:22 - INFO - lib.train_eval -     Num examples = 872
03/21/2019 17:26:22 - INFO - lib.train_eval -     Batch size = 8

Evaluating: 100%|██████████| 109/109 [00:00<00:00, 416.47it/s]03/21/2019 17:26:23 - INFO - lib.train_eval -   ***** Eval results *****
03/21/2019 17:26:23 - INFO - lib.train_eval -     eval_accuracy = 0.7844036697247706
03/21/2019 17:26:23 - INFO - lib.train_eval -     eval_loss = -0.00040045435275506535


{'eval_loss': -0.00040045435275506535, 'eval_accuracy': 0.7844036697247706}

Всего получилось где-то 20 эпох

Теперь посмотрим на ту же модель с ***8 модулями*** свертки.

In [47]:
params = {
    'data_dir': '../../data/SST-2',
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'task_name': 'sst2',
    'bert_model': 'bert-base-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-3,
    'warmup_proportion': 0.1,
    'num_train_epochs': 15,
    'seed': 1331
}

model = BlendCNN(params['max_seq_length'], len(tokenizer.vocab), len(label_list), n_layers=8).to(device)
model, result = train_student(model, teacher_model, train_examples, label_list, params, tokenizer)

***** Running training *****
Num examples: 67349
Batch size:   32
Num steps:    31560


Evaluating:  24%|██▎       | 496/2105 [00:46<02:32, 10.58it/s]


Evaluating:  47%|████▋     | 992/2105 [01:33<01:46, 10.50it/s]


Evaluating:  70%|███████   | 1480/2105 [02:20<00:59, 10.43it/s]


Evaluating:  93%|█████████▎| 1968/2105 [03:06<00:13, 10.43it/s]


...


Iteration:   0%|          | 10/2105 [00:00<00:20, 99.89it/s]

Epoch: 14



Iteration:   1%|          | 11/2105 [00:00<00:20, 100.14it/s]

Epoch: 15



Iteration: 100%|██████████| 2105/2105 [00:20<00:00, 101.56it/s]

In [48]:
evaluate(model, eval_examples, label_list, params, tokenizer)

03/21/2019 17:45:26 - INFO - lib.train_eval -   ***** Running evaluation *****
03/21/2019 17:45:26 - INFO - lib.train_eval -     Num examples = 872
03/21/2019 17:45:26 - INFO - lib.train_eval -     Batch size = 8

Evaluating: 100%|██████████| 109/109 [00:00<00:00, 331.13it/s]03/21/2019 17:45:26 - INFO - lib.train_eval -   ***** Eval results *****
03/21/2019 17:45:26 - INFO - lib.train_eval -     eval_accuracy = 0.7964036697247706
03/21/2019 17:45:26 - INFO - lib.train_eval -     eval_loss = -0.023476406231249143


{'eval_loss': -0.023476406231249143, 'eval_accuracy': 0.7964036697247706}

In [53]:
all_count = 0
for p in model.named_parameters():
    cur_count = 1
    for sh in p[1].size():
        cur_count *= sh
        
    all_count += cur_count
    
print('compresing in {} times'.format(109483778 / all_count))

compresing in 23.24299942382788 times
